## 3 signal stacking

In [1]:
import numpy as np
import pandas as pd

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

/Users/louisalu/Library/Python/3.8/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/louisalu/Documents/04WSP/venv/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
#!pip install bokeh==2.4.2

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df_price = (
    pd.read_csv('dowj_price_indicators.csv', index_col=0)
    .rename(columns={'Close*':'Close'})
    [['Date', 'Open', 'High', 'Low', 'Close']]
    .assign(Date = lambda df: pd.to_datetime(df.Date))
    .sort_values('Date')
    .set_index('Date', drop=True)
)

In [4]:
df_price

,Open,High,Low,Close
Date,,,,
2008-09-17,11056.58,11057.31,10595.90,10609.66
2008-09-18,10609.01,11076.44,10459.44,11019.69
2008-09-19,11027.51,11483.05,11026.70,11388.44
2008-09-22,11394.42,11394.58,10992.20,11015.69
2008-09-23,11015.69,11143.21,10833.94,10854.17
...,...,...,...,...
2016-06-27,17355.21,17355.21,17063.08,17140.24
2016-06-28,17190.51,17409.72,17190.51,17409.72
2016-06-29,17456.02,17704.51,17456.02,17694.68


In [5]:
df_predictions = (
    pd.read_csv('prediction_output.csv', index_col=0)
    .assign(Date = lambda df: pd.to_datetime(df.Date))
    .sort_values('Date')
    .set_index('Date', drop=True)
)

In [6]:
df_predictions

,iterate_prediction,grid_prediction,random_prediction
Date,,,
2014-03-05,-0.000488,0.000614,-0.002610
2014-03-06,0.004805,0.002387,-0.000320
2014-03-07,0.001993,0.002387,-0.000598
2014-03-10,0.003717,0.001717,-0.001000
2014-03-11,-0.001528,0.001017,-0.000465
...,...,...,...
2016-06-27,0.035532,0.005833,0.004592
2016-06-28,0.022988,0.012130,0.011819
2016-06-29,0.026804,0.005521,0.003258


In [7]:
# Predictions includes only testing data so the resulting df has only that as well
df_price_and_predictions = df_predictions.join(df_price)
df_price_and_predictions

,iterate_prediction,grid_prediction,random_prediction,Open,High,Low,Close
Date,,,,,,,
2014-03-05,-0.000488,0.000614,-0.002610,16395.88,16406.55,16343.96,16360.18
2014-03-06,0.004805,0.002387,-0.000320,16360.56,16450.17,16360.56,16421.89
2014-03-07,0.001993,0.002387,-0.000598,16424.53,16505.70,16398.86,16452.72
2014-03-10,0.003717,0.001717,-0.001000,16453.10,16453.10,16334.20,16418.68
2014-03-11,-0.001528,0.001017,-0.000465,16419.39,16460.33,16325.17,16351.25
...,...,...,...,...,...,...,...
2016-06-27,0.035532,0.005833,0.004592,17355.21,17355.21,17063.08,17140.24
2016-06-28,0.022988,0.012130,0.011819,17190.51,17409.72,17190.51,17409.72
2016-06-29,0.026804,0.005521,0.003258,17456.02,17704.51,17456.02,17694.68


In [8]:
def identity(x):
    '''Identity'''
    return x

class SignalStackingStrat1(Strategy):
    """
    Opens a long position whenever 2 or more prediction values
    are positive. Close it otherwise.
    """

    def init(self):
        self.p1 = self.I(identity, self.data.random_prediction)
        self.p2 = self.I(identity, self.data.iterate_prediction)
        self.p3 = self.I(identity, self.data.grid_prediction)

    def next(self):
        # Positive signals
        ps1 = int(self.p1[-1] >= 0)
        ps2 = int(self.p2[-1] >= 0)
        ps3 = int(self.p3[-1] >= 0)

        if (ps1 + ps2 + ps3) >= 2:
            self.buy()
        else:
            self.position.close()


In [9]:
# Creates backtest obj, runs and plots
bt = Backtest(df_price_and_predictions, SignalStackingStrat1, cash=100_000_000)#, exclusive_orders=True)
print(bt.run())
bt.plot()

Start                     2014-03-05 00:00:00
End                       2016-07-01 00:00:00
Duration                    849 days 00:00:00
Exposure Time [%]                   73.469388
Equity Final [$]                 112374609.33
Equity Peak [$]                  113294203.33
Return [%]                          12.374609
Buy & Hold Return [%]                9.713768
Return (Ann.) [%]                    5.127162
Volatility (Ann.) [%]               12.056266
Sharpe Ratio                         0.425269
Sortino Ratio                        0.639735
Calmar Ratio                         0.530425
Max. Drawdown [%]                   -9.666147
Avg. Drawdown [%]                   -2.331028
Max. Drawdown Duration      164 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                  108
Win Rate [%]                        58.333333
Best Trade [%]                       3.587229
Worst Trade [%]                     -5.049272
Avg. Trade [%]                    

Row(id='1555', ...)

In [10]:
def identity(x):
    '''Identity'''
    return x

class SignalStackingStrat2(Strategy):
    """
    Open a long position whenever the 3 prediction values are positive.
    Open a short position whenever the 3 prediction values are negative.
    """

    def init(self):
        self.p1 = self.I(identity, self.data.random_prediction)
        self.p2 = self.I(identity, self.data.iterate_prediction)
        self.p3 = self.I(identity, self.data.grid_prediction)

    def next(self):
        # Positive signals
        ps1 = int(self.p1[-1] >= 0)
        ps2 = int(self.p2[-1] >= 0)
        ps3 = int(self.p3[-1] >= 0)

        # Negative signals
        ns1 = int(self.p1[-1] < 0)
        ns2 = int(self.p2[-1] < 0)
        ns3 = int(self.p3[-1] < 0)

        if (ps1 + ps2 + ps3) == 3:
            self.buy()
        elif (ns1 + ns2 + ns3) == 3:
            self.sell()


In [11]:
# Creates backtest obj, runs and plots
bt = Backtest(df_price_and_predictions, SignalStackingStrat2, cash=100_000_000, exclusive_orders=True)
print(bt.run())
bt.plot()

Start                     2014-03-05 00:00:00
End                       2016-07-01 00:00:00
Duration                    849 days 00:00:00
Exposure Time [%]                   97.959184
Equity Final [$]                 125937480.41
Equity Peak [$]                  128048092.25
Return [%]                           25.93748
Buy & Hold Return [%]                9.713768
Return (Ann.) [%]                   10.388434
Volatility (Ann.) [%]                15.22537
Sharpe Ratio                         0.682311
Sortino Ratio                        1.111478
Calmar Ratio                         0.884584
Max. Drawdown [%]                  -11.743865
Avg. Drawdown [%]                   -2.521663
Max. Drawdown Duration      251 days 00:00:00
Avg. Drawdown Duration       34 days 00:00:00
# Trades                                  247
Win Rate [%]                        54.251012
Best Trade [%]                       3.043256
Worst Trade [%]                     -4.640792
Avg. Trade [%]                    

Row(id='2512', ...)

## how to start your own project

In [ ]:
#!pip install yfinance

In [1]:
import yfinance as yf


In [2]:
msft = yf.Ticker("MSFT")


In [4]:
msft.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'fax': '425 706 7329',
 'website': 'https://www.microsoft.com',
 'industry': 'Software—Infrastructure',
 'industryDisp': 'Software—Infrastructure',
 'sector': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. The company operates in three segments: Productivity and Business Processes, Intelligent Cloud, and More Personal Computing. The Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, Microsoft Viva, and Skype for Business; Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. The Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, Syst

In [9]:
hist = msft.history(period="120mo")
display(hist)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2013-07-29 00:00:00-04:00,26.224441,26.332772,26.166109,26.282774,28870700,0.0,0.0
2013-07-30 00:00:00-04:00,26.482773,26.766099,26.291110,26.541105,45799500,0.0,0.0
2013-07-31 00:00:00-04:00,26.641099,26.707764,26.424436,26.532768,43898400,0.0,0.0
2013-08-01 00:00:00-04:00,26.716100,26.741098,26.332774,26.391106,42557900,0.0,0.0
2013-08-02 00:00:00-04:00,26.407770,26.582766,26.307772,26.574432,29199900,0.0,0.0
...,...,...,...,...,...,...,...
2023-07-24 00:00:00-04:00,345.850006,346.920013,342.309998,345.109985,26678100,0.0,0.0
2023-07-25 00:00:00-04:00,347.109985,351.890015,345.070007,350.980011,41637700,0.0,0.0
2023-07-26 00:00:00-04:00,341.440002,344.670013,333.109985,337.769989,58383700,0.0,0.0


In [10]:
# show meta information about the history (requires history() to be called first)
msft.history_metadata


{'currency': 'USD',
 'symbol': 'MSFT',
 'exchangeName': 'NMS',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 511108200,
 'regularMarketTime': 1690574401,
 'gmtoffset': -14400,
 'timezone': 'EDT',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 338.37,
 'chartPreviousClose': 31.62,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EDT',
   'end': 1690551000,
   'start': 1690531200,
   'gmtoffset': -14400},
  'regular': {'timezone': 'EDT',
   'end': 1690574400,
   'start': 1690551000,
   'gmtoffset': -14400},
  'post': {'timezone': 'EDT',
   'end': 1690588800,
   'start': 1690574400,
   'gmtoffset': -14400}},
 'dataGranularity': '1d',
 'range': '120mo',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [12]:
display(msft.actions)


,Dividends,Stock Splits
Date,,
2013-08-13 00:00:00-04:00,0.23,0.0
2013-11-19 00:00:00-05:00,0.28,0.0
2014-02-18 00:00:00-05:00,0.28,0.0
2014-05-13 00:00:00-04:00,0.28,0.0
2014-08-19 00:00:00-04:00,0.28,0.0
2014-11-18 00:00:00-05:00,0.31,0.0
2015-02-17 00:00:00-05:00,0.31,0.0
2015-05-19 00:00:00-04:00,0.31,0.0
2015-08-18 00:00:00-04:00,0.31,0.0


In [13]:
msft.dividends



Date
2013-08-13 00:00:00-04:00    0.23
2013-11-19 00:00:00-05:00    0.28
2014-02-18 00:00:00-05:00    0.28
2014-05-13 00:00:00-04:00    0.28
2014-08-19 00:00:00-04:00    0.28
2014-11-18 00:00:00-05:00    0.31
2015-02-17 00:00:00-05:00    0.31
2015-05-19 00:00:00-04:00    0.31
2015-08-18 00:00:00-04:00    0.31
2015-11-17 00:00:00-05:00    0.36
2016-02-16 00:00:00-05:00    0.36
2016-05-17 00:00:00-04:00    0.36
2016-08-16 00:00:00-04:00    0.36
2016-11-15 00:00:00-05:00    0.39
2017-02-14 00:00:00-05:00    0.39
2017-05-16 00:00:00-04:00    0.39
2017-08-15 00:00:00-04:00    0.39
2017-11-15 00:00:00-05:00    0.42
2018-02-14 00:00:00-05:00    0.42
2018-05-16 00:00:00-04:00    0.42
2018-08-15 00:00:00-04:00    0.42
2018-11-14 00:00:00-05:00    0.46
2019-02-20 00:00:00-05:00    0.46
2019-05-15 00:00:00-04:00    0.46
2019-08-14 00:00:00-04:00    0.46
2019-11-20 00:00:00-05:00    0.51
2020-02-19 00:00:00-05:00    0.51
2020-05-20 00:00:00-04:00    0.51
2020-08-19 00:00:00-04:00    0.51
2020-11-1

In [14]:
msft.splits

Series([], Name: Stock Splits, dtype: float64)

In [15]:
# show share count
msft.get_shares_full(start="2022-01-01", end=None)


2022-01-27 00:00:00-05:00    7496869888
2022-02-04 00:00:00-05:00    7800719872
2022-02-05 00:00:00-05:00    7496869888
2022-02-11 00:00:00-05:00    7496869888
2022-03-04 00:00:00-05:00    7605040128
                                ...    
2023-07-07 00:00:00-04:00    7435489792
2023-07-12 00:00:00-04:00    7449829888
2023-07-13 00:00:00-04:00    7435489792
2023-07-19 00:00:00-04:00    7731420160
2023-07-21 00:00:00-04:00    7435489792
Length: 61, dtype: int64

In [16]:
# - income statement
msft.income_stmt



,2023-06-30,2022-06-30,2021-06-30,2020-06-30
Tax Effect Of Unusual Items,0.0,0.0,0.0,0.0
Tax Rate For Calcs,0.189786,0.131134,0.138266,0.16
Normalized EBITDA,102384000000.0,97843000000.0,81602000000.0,65755000000.0
Total Unusual Items,NaN,334000000.0,1303000000.0,28000000.0
Total Unusual Items Excluding Goodwill,NaN,334000000.0,1303000000.0,28000000.0
Net Income From Continuing Operation Net Minority Interest,72361000000.0,72738000000.0,61271000000.0,44281000000.0
Reconciled Depreciation,13861000000.0,14460000000.0,11686000000.0,12796000000.0
Reconciled Cost Of Revenue,65863000000.0,62650000000.0,52232000000.0,46078000000.0
EBIT,88523000000.0,83383000000.0,69916000000.0,52959000000.0
Net Interest Income,NaN,31000000.0,-215000000.0,89000000.0


In [17]:
msft.quarterly_income_stmt

,2023-06-30,2023-03-31,2022-12-31,2022-09-30,2022-06-30
Tax Effect Of Unusual Items,0.0,0.0,0.0,-10640000.0,0.0
Tax Rate For Calcs,0.19,0.192917,0.192438,0.19,0.182896
Normalized EBITDA,28128000000.0,25901000000.0,24047000000.0,24918000000.0,24513000000.0
Total Unusual Items,NaN,162000000.0,-232000000.0,-56000000.0,-80000000.0
Total Unusual Items Excluding Goodwill,NaN,162000000.0,-232000000.0,-56000000.0,-80000000.0
Net Income From Continuing Operation Net Minority Interest,20081000000.0,18299000000.0,16425000000.0,17556000000.0,16740000000.0
Reconciled Depreciation,3874000000.0,3549000000.0,3648000000.0,2790000000.0,3979000000.0
Reconciled Cost Of Revenue,16795000000.0,16128000000.0,17488000000.0,15452000000.0,16429000000.0
EBIT,24254000000.0,22352000000.0,20399000000.0,22072000000.0,20534000000.0
Net Interest Income,NaN,252000000.0,210000000.0,141000000.0,56000000.0


In [18]:
# show holders
display(msft.major_holders)
display(msft.institutional_holders)
display(msft.mutualfund_holders)


,0,1
0,0.05%,% of Shares Held by All Insider
1,73.77%,% of Shares Held by Institutions
2,73.80%,% of Float Held by Institutions
3,6159,Number of Institutions Holding Shares


,Holder,Shares,Date Reported,% Out,Value
0,Vanguard Group Inc,649544303,2023-03-30,0.0874,219786302634
1,Blackrock Inc.,537573096,2023-03-30,0.0723,181898605868
2,State Street Corporation,292106885,2023-03-30,0.0393,98840205251
3,"FMR, LLC",200523373,2023-03-30,0.0270,67851092742
4,Price (T.Rowe) Associates Inc,166035734,2023-03-30,0.0223,56181510502
5,"Geode Capital Management, LLC",144365760,2023-03-30,0.0194,48849041506
6,Morgan Stanley,120316895,2023-03-30,0.0162,40711627173
7,JP Morgan Chase & Company,94617741,2023-03-30,0.0127,32015804560
8,Capital World Investors,87608980,2023-03-30,0.0118,29644250134
9,Norges Bank Investment Management,86316926,2022-12-30,0.0116,29207057829


,Holder,Shares,Date Reported,% Out,Value
0,Vanguard Total Stock Market Index Fund,226594602,2022-12-30,0.0304,76672814372
1,Vanguard 500 Index Fund,172326589,2022-12-30,0.0232,58310147078
2,Fidelity 500 Index Fund,81720092,2023-02-27,0.0110,27651627131
3,SPDR S&P 500 ETF Trust,80591530,2023-03-30,0.0108,27269755612
4,"Invesco ETF Tr-Invesco QQQ Tr, Series 1 ETF",75130340,2023-03-30,0.0101,25421852778
5,iShares Core S&P 500 ETF,66532494,2023-02-27,0.0089,22512599669
6,Vanguard Growth Index Fund,62920006,2022-12-30,0.0085,21290242122
7,Vanguard Institutional Index Fund-Institutiona...,52237514,2022-12-30,0.0070,17675607357
8,Growth Fund Of America Inc,41256784,2023-03-30,0.0055,13960057800
9,Vanguard Information Technology Index Fund,34018479,2022-11-29,0.0046,11510832573


In [19]:
# show news
msft.news

[{'uuid': 'b3fa25dc-2b85-3119-9d25-c0e3a7639344',
  'title': 'Dow Jones Futures Fall After Market Rally Powers Higher; Apple, Tesla Lead Stocks To Watch',
  'publisher': "Investor's Business Daily",
  'link': 'https://finance.yahoo.com/m/b3fa25dc-2b85-3119-9d25-c0e3a7639344/dow-jones-futures-fall-after.html',
  'providerPublishTime': 1690775335,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/1Dv8tjENy.cqs9vAmCgNKQ--~B/aD01NjM7dz0xMDAwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/ibd.com/7009328d95c23df6351c4a1d33db531c',
     'width': 1000,
     'height': 563,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/T9vbQ1XV.1cuYNz97Gj7.A--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/ibd.com/7009328d95c23df6351c4a1d33db531c',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['TSLA',
   'COMP',
   '^GSPC',
   '^DJI',
   'MRVL',
   'AAPL',
   'MSFT

In [20]:
#multiple tickers

In [21]:
tickers = yf.Tickers('msft aapl goog')


In [22]:
tickers

yfinance.Tickers object <MSFT,AAPL,GOOG>

In [23]:
tickers.tickers['MSFT'].info


{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'fax': '425 706 7329',
 'website': 'https://www.microsoft.com',
 'industry': 'Software—Infrastructure',
 'industryDisp': 'Software—Infrastructure',
 'sector': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. The company operates in three segments: Productivity and Business Processes, Intelligent Cloud, and More Personal Computing. The Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, Microsoft Viva, and Skype for Business; Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. The Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, Syst

In [24]:
tickers.tickers['AAPL'].history(period="1mo")


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-06-29 00:00:00-04:00,189.080002,190.070007,188.940002,189.589996,46347300,0.0,0.0
2023-06-30 00:00:00-04:00,191.630005,194.479996,191.259995,193.970001,85069600,0.0,0.0
2023-07-03 00:00:00-04:00,193.779999,193.880005,191.759995,192.460007,31458200,0.0,0.0
2023-07-05 00:00:00-04:00,191.570007,192.979996,190.619995,191.330002,46920300,0.0,0.0
2023-07-06 00:00:00-04:00,189.839996,192.020004,189.199997,191.809998,45094300,0.0,0.0
2023-07-07 00:00:00-04:00,191.410004,192.669998,190.240005,190.679993,46778000,0.0,0.0
2023-07-10 00:00:00-04:00,189.259995,189.990005,187.039993,188.610001,59922200,0.0,0.0
2023-07-11 00:00:00-04:00,189.160004,189.300003,186.600006,188.080002,46638100,0.0,0.0
2023-07-12 00:00:00-04:00,189.679993,191.699997,188.470001,189.770004,60750200,0.0,0.0


In [25]:
tickers.tickers['GOOG'].actions


,Dividends,Stock Splits
Date,,
2014-03-27 00:00:00-04:00,0.0,2.002000
2015-04-27 00:00:00-04:00,0.0,1.002746
2022-07-18 00:00:00-04:00,0.0,20.000000


In [ ]:
# to download the price data

In [26]:
data = yf.download("SPY AAPL", period="1mo")


[*********************100%***********************]  2 of 2 completed


In [27]:
data

Adj Close                   Close                    High  \
                  AAPL         SPY        AAPL         SPY        AAPL   
Date                                                                     
2023-06-29  189.589996  438.109985  189.589996  438.109985  190.070007   
2023-06-30  193.970001  443.279999  193.970001  443.279999  194.479996   
2023-07-03  192.460007  443.790009  192.460007  443.790009  193.880005   
2023-07-05  191.330002  443.130005  191.330002  443.130005  192.979996   
2023-07-06  191.809998  439.660004  191.809998  439.660004  192.020004   
2023-07-07  190.679993  438.549988  190.679993  438.549988  192.669998   
2023-07-10  188.610001  439.660004  188.610001  439.660004  189.990005   
2023-07-11  188.080002  442.459991  188.080002  442.459991  189.300003   
2023-07-12  189.770004  446.019989  189.770004  446.019989  191.699997   
2023-07-13  190.539993  449.559998  190.539993  449.559998  191.190002   
2023-07-14  190.690002  449.279999  190.690002  449.279999  191.179993   
2023-07-17  193.990005  450.839996  193.990005  450.839996  194.320007   
2023-07-18  193.729996  454.190002  193.729996  454.190002  194.330002   
2023-07-19  195.100006  455.200012  195.100006  455.200012  198.229996   
2023-07-20  193.130005  452.179993  193.130005  452.179993  196.470001   
2023-07-21  191.940002  452.179993  191.940002  452.179993  194.970001   
2023-07-24  192.750000  454.200012  192.750000  454.200012  194.910004   
2023-07-25  193.619995  455.440002  193.619995  455.440002  194.440002   
2023-07-26  194.500000  455.510010  194.500000  455.510010  195.639999   
2023-07-27  193.220001  452.489990  193.220001  452.489990  197.199997   
2023-07-28  195.830002  456.920013  195.830002  456.920013  196.630005   

                               Low                    Open              \
                   SPY        AAPL         SPY        AAPL         SPY   
Date                                                                     
2023-06-29  438.279999  188.940002  435.540009  189.080002  435.959991   
2023-06-30  444.299988  191.259995  441.109985  191.630005  441.440002   
2023-07-03  444.079987  191.759995  442.630005  193.779999  442.920013   
2023-07-05  443.890015  190.619995  441.899994  191.570007  441.910004   
2023-07-06  440.100006  189.199997  437.059998  189.839996  439.420013   
2023-07-07  442.640015  190.240005  438.299988  191.410004  438.630005   
2023-07-10  439.839996  187.039993  437.589996  189.259995  438.179993   
2023-07-11  442.970001  186.600006  439.440002  189.160004  440.450012   
2023-07-12  447.480011  188.470001  444.910004  189.679993  446.390015   
2023-07-13  450.380005  189.779999  447.450012  190.500000  447.899994   
2023-07-14  451.359985  189.630005  448.489990  190.229996  450.480011   
2023-07-17  451.929993  191.809998  449.079987  191.899994  449.130005   
2023-07-18  454.859985  192.419998  450.049988  193.350006  450.500000   
2023-07-19  456.429993  192.649994  454.109985  193.100006  455.010010   
2023-07-20  455.100006  192.500000  451.440002  195.089996  454.170013   
2023-07-21  454.170013  191.229996  452.170013  194.100006  453.959991   
2023-07-24  455.040009  192.250000  452.299988  193.410004  453.369995   
2023-07-25  456.739990  192.919998  453.869995  193.330002  453.920013   
2023-07-26  456.989990  193.320007  453.380005  193.669998  454.470001   
2023-07-27  459.440002  192.550003  451.549988  196.020004  459.019989   
2023-07-28  457.779999  194.139999  452.489990  194.669998  455.880005   

              Volume             
                AAPL        SPY  
Date                             
2023-06-29  46347300   67882300  
2023-06-30  85069600  104921500  
2023-07-03  31458200   32793400  
2023-07-05  46920300   58418400  
2023-07-06  45094300   80658300  
2023-07-07  46778000   86076100  
2023-07-10  59922200   62443500  
2023-07-11  46638100   64463800  
2023-07-12  60750200   91924500  
2023-07-13  41342300   72425200  
2023-07-14  41573